# Juntando as Peças 🧩

## (Fontes de Dados 🚰 ➕ Limpeza e Preparação dos Dados 🏗️ + Tratamento dos Dados 🪢 + Agregação dos Dados 🎒 + Visualização de Dados 🔍) <sup>Alto Desempenho 🚀 🧞‍♂️</sup>

In [1]:
!pip install --upgrade geojson_rewind --quiet


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import pickle
import os
import pandas as pd
import requests
import plotly.express as px
import plotly.io as pio
import math
from geojson_rewind import rewind
pio.renderers.default = 'iframe'

### Funções Auxiliares

#### Dados de Carregamento e Salvamento de Dados/Pickle

In [3]:
def build_path(subfolder = 'merge'):
    folderpath = os.path.join(os.getcwd(), os.pardir, 
                              'project', 'data', subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): 
        os.makedirs(folderpath)
    return folderpath

def load_data():
    data = {}   
    folderpath = build_path()
    for filename in os.listdir(folderpath):
        if filename.endswith('.json'):
            filepath = os.path.join(build_path('merge'), filename)
            data[filename.split('.')[0]] = pd.read_json(filepath)
    return data

def save_data(figure, type_of_map, intraregion, subfolder = 'app'):
    filepath = os.path.join(build_path(subfolder), f'{type_of_map}-{intraregion}.pkl')
    
    with open(filepath, 'wb') as pickle_file:
        pickle.dump(figure, pickle_file)      

#### Dados Geográficos

In [4]:
def get_central_geolocation(df):
    latitudes = df.groupby('latitude').agg({'latitude': 'first'})['latitude'].apply(math.radians).reset_index(drop=True)
    longitudes = df.groupby('longitude').agg({'longitude': 'first'})['longitude'].apply(math.radians).reset_index(drop=True)

    x = (latitudes.apply(math.cos) * longitudes.apply(math.cos)).sum() / latitudes.size
    y = (latitudes.apply(math.cos) * longitudes.apply(math.sin)).sum() / latitudes.size
    z = latitudes.apply(math.sin).sum() / latitudes.size

    central_longitude = math.atan2(y, x)
    central_latitude = math.atan2(z, (x**2 + y**2)**0.5)
    
    return math.degrees(central_longitude), math.degrees(central_latitude)
    
def get_region_information(df):
    central_longitude, central_latitude = get_central_geolocation(df)
    cases = df['casosNovos'].sum()
    deaths = df['obitosNovos'].sum()
    
    return pd.Series(data=[central_longitude, central_latitude, cases, deaths], 
                     index=['longitude', 'latitude', 'casos', 'obitos'])

#### Desenhos

In [5]:
#API de malhas geográficas
#https://servicodados.ibge.gov.br/api/docs/malhas?versao=3

#API de localidades
#https://servicodados.ibge.gov.br/api/docs/localidades

In [6]:
def plot_helper(city_infection, intraregion):
    code_length = {'regiao': 1, 'mesorregiao': 4, 'microrregiao': 5, 'municipio': 6}.get(intraregion)
    intraregion_plural = {'regiao': 'regioes', 'mesorregiao': 'mesorregioes',
                          'microrregiao': 'microrregioes', 'municipio': 'municipios'}.get(intraregion)
    
    city_infection[f'id.{intraregion_plural}'] = city_infection['codigo_ibge'].astype(str).apply(lambda x: x[:code_length])
    intraregion_plural_names = pd.json_normalize(requests.get(f'http://servicodados.ibge.gov.br/api/v1/localidades/{intraregion_plural}').json())[['id', 'nome']]
    intraregion_plural_names['id'] = intraregion_plural_names['id'].astype(str)

    return city_infection, code_length, intraregion_plural, intraregion_plural_names

In [7]:
def plot_scatter_geo(city_infection, intraregion = 'regiao'):
    city_infection, code_length, intraregion_plural, intraregion_plural_names = \
        plot_helper(city_infection, intraregion)

    if intraregion == 'municipio':
        intraregion_plural_names['id'] = intraregion_plural_names['id'].apply(lambda x: x[:code_length])

    fig = px.scatter_geo(city_infection.groupby(f'id.{intraregion_plural}').\
                             apply(get_region_information).\
                                 merge(intraregion_plural_names, 
                                       left_index=True, 
                                       right_on='id'),
                         lat='latitude', 
                         lon='longitude',
                         hover_name='nome',
                         size='obitos',
                         scope='south america')
    fig.show();
    
    return fig

In [8]:
def plot_choroplet(city_infection, intraregion = 'regiao'):

    city_infection, code_length, intraregion_plural, intraregion_plural_names = \
        plot_helper(city_infection, intraregion)
    
    geojson = requests.get(f'http://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=maxima&intrarregiao={intraregion}').json()
    
    if intraregion == 'municipio':
        intraregion_plural_names['id'] = intraregion_plural_names['id'].apply(lambda x: x[:code_length])
        for feature in geojson['features']:
            feature['properties']['codarea'] = feature['properties']['codarea'][:code_length]
    geojson = rewind(geojson, rfc7946=False)
    
    fig = px.choropleth(city_infection.groupby(f'id.{intraregion_plural}').\
                             apply(get_region_information).\
                                 merge(intraregion_plural_names, 
                                       left_index=True, 
                                       right_on='id'),
                        geojson=geojson,
                        scope='south america',
                        color='obitos',
                        color_continuous_scale="Reds",
                        locations='id',
                        featureidkey='properties.codarea',
                        hover_name='nome')

    fig.show()
    
    return fig

### Código

In [9]:
data = load_data()

In [10]:
figure = plot_scatter_geo(data['city_infecction'], 'regiao')
save_data(figure, 'scatter_geo', 'regiao')

In [11]:
figure = plot_scatter_geo(data['city_infecction'], 'mesorregiao')
save_data(figure, 'scatter_geo', 'mesorregiao')

In [12]:
figure = plot_scatter_geo(data['city_infecction'], 'microrregiao')
save_data(figure, 'scatter_geo', 'microrregiao')

In [13]:
figure = plot_scatter_geo(data['city_infecction'], 'municipio')
save_data(figure, 'scatter_geo', 'municipio')

In [14]:
figure = plot_choroplet(data['city_infecction'], 'regiao')
save_data(figure, 'choropleth', 'regiao')

In [15]:
figure = plot_choroplet(data['city_infecction'], 'mesorregiao')
save_data(figure, 'choropleth', 'mesorregiao')

In [16]:
figure = plot_choroplet(data['city_infecction'], 'microrregiao')
save_data(figure, 'choropleth', 'microrregiao')

In [17]:
figure = plot_choroplet(data['city_infecction'], 'municipio')
save_data(figure, 'choropleth', 'municipio')